libraries that we need

In [1]:
import os
import zipfile
import pandas as pd
from collections import defaultdict
import re
import matplotlib.pyplot as plt

<h1>
extracting data from a zip file
</h1>

In [ ]:
rar_file_path = r"F:\daneshkar\project1\202405.zip" 
output_folder = 'extracted_rar_files'  

if not os.path.exists(output_folder):
    os.makedirs(output_folder)


with zipfile.ZipFile(rar_file_path, 'r') as rf:
    rf.extractall(output_folder)


for filename in os.listdir(output_folder):
    file_path = os.path.join(output_folder, filename)
    if filename.endswith('.zip'):
        print(f'extracting file {filename} ')
        with zipfile.ZipFile(file_path, 'r') as rf_inner:
            rf_inner.extractall(os.path.join(output_folder, filename[:-4])) 

read parquet files and create a dataframe and save them in csv files

In [ ]:
import os
import pandas as pd
import re
from collections import defaultdict

def read_and_group_parquet_files(base_folder, start_date, end_date, folders_in_row):
    data_frames = defaultdict(list)
    group_names = [
        'home_team_score',
        'home_team',
        'away_team',
        'away_team_score',
        'event',
        'round',
        'season',
        'time',
        'tournament',
        'venue',
        'odds',
        'pbp',
        'statistics',
        'power',
        'votes'
    ]

    # Regular expressions for away_team and away_team_score
    away_team_pattern = re.compile(r'^away_team_')
    away_team_score_pattern = re.compile(r'^away_team_score_')

    for date in range(start_date, end_date + 1):
        for folder in folders_in_row:
            date_folder = os.path.join(base_folder, str(date), 'data', 'raw', folder)
            print(f'Reading folder: {date_folder}')  # Monitor the current folder
            if os.path.isdir(date_folder):
                for file_name in os.listdir(date_folder):
                    if file_name.endswith('.parquet'):
                        matched_group = None
                        # Check each group name if the filename starts with it
                        if away_team_score_pattern.match(file_name):
                            matched_group = 'away_team_score'
                        elif away_team_pattern.match(file_name):
                            matched_group = 'away_team'
                        else:
                            # Simple string matching for other group names
                            for group_name in group_names:
                                if file_name.startswith(group_name + '_'):
                                    matched_group = group_name
                                    break

                        if matched_group:
                            file_path = os.path.join(date_folder, file_name)
                            df = pd.read_parquet(file_path)
                            data_frames[matched_group].append(df)

    # Concatenate DataFrames in each group and save to CSV
    grouped_data_frames = {}
    for group, dfs in data_frames.items():
        if dfs:  # Check if the list of DataFrames is not empty
            concatenated_df = pd.concat(dfs, ignore_index=True)
            grouped_data_frames[group] = concatenated_df
            # Save DataFrame to CSV
            csv_file_path = os.path.join(base_folder, f'{group}.csv')
            concatenated_df.to_csv(csv_file_path, index=False)
            print(f'Saved {group} DataFrame to {csv_file_path}')

    return grouped_data_frames

# Example usage
base_folder = r'F:\daneshkar\project1\extracted_rar_files'
start_date = 20240501
end_date = 20240531
folders_in_row = (
    'raw_match_parquet',
    'raw_odds_parquet',
    'raw_point_by_point_parquet',
    'raw_statistics_parquet',
    'raw_tennis_power_parquet',
    'raw_votes_parquet'
)

grouped_data_frames = read_and_group_parquet_files(base_folder, start_date, end_date, folders_in_row)

# Display the groups and their DataFrames
for group_name, df in grouped_data_frames.items():
    print(f'Group: {group_name}')
    # print(df.head())  # Print the first few rows of each DataFrame


read csv files

In [ ]:
# Define the directory and group names
base_folder = r'F:\daneshkar\project1\extracted_rar_files'
group_names = [
    'home_team_score',
    'home_team',
    'away_team',
    'away_team_score',
    'event',
    'round',
    'season',
    'time',
    'tournament',
    'venue',
    'odds',
    'pbp',
    'statistics',
    'power',
    'votes'
]

# Initialize a dictionary to store the DataFrames
grouped_data_frames = {}

# Read each CSV file into a DataFrame and store it in the dictionary
for group_name in group_names:
    csv_file_path = os.path.join(base_folder, f'{group_name}.csv')
    if os.path.exists(csv_file_path):
        df = pd.read_csv(csv_file_path)
        grouped_data_frames[group_name] = df
        print(f'Successfully read {group_name}.csv')
    else:
        print(f'File {group_name}.csv not found')

# Display the DataFrames (or perform further operations)
for group_name, df in grouped_data_frames.items():
    print(f'Group: {group_name}')
    # print(df.head())  # Print the first few rows of each DataFrame


check the shape of dataframes

In [ ]:
# Display the shape of each DataFrame
for group_name, df in grouped_data_frames.items():
    print(f'Group: {group_name}, Shape: {df.shape}')

1. How many tennis players are included in the dataset?

In [ ]:
def count_unique_players(grouped_data_frames):
    player_ids = set()
    
    # Extract player IDs from home team info
    if 'home_team' in grouped_data_frames:
        home_team_df = grouped_data_frames['home_team']
        if 'player_id' in home_team_df.columns:
            player_ids.update(home_team_df['player_id'].unique())
    
    # Extract player IDs from away team info
    if 'away_team' in grouped_data_frames:
        away_team_df = grouped_data_frames['away_team']
        if 'player_id' in away_team_df.columns:
            player_ids.update(away_team_df['player_id'].unique())
    
    # Return the count of unique player IDs
    return len(player_ids)

# Count unique players
num_unique_players = count_unique_players(grouped_data_frames)
print(f'The number of unique tennis players in the dataset is: {num_unique_players}')


otherway

In [ ]:
event = grouped_data_frames['event']
home_team = grouped_data_frames['home_team']
away_team = grouped_data_frames['away_team']

# Combine home and away team info into one DataFrame
players_info = pd.concat([
    home_team[['player_id', 'full_name']],
    away_team[['player_id', 'full_name']]
]).drop_duplicates().reset_index(drop=True)
number_of_tennis_players=len(players_info)

print(f'The number of unique tennis players in the dataset is: {number_of_tennis_players}')

3.Which player has the highest number of wins?

In [ ]:
import pandas as pd

# Assuming grouped_data_frames is your dictionary containing all the dataframes
event = grouped_data_frames['event']
home_team = grouped_data_frames['home_team']
away_team = grouped_data_frames['away_team']

# Combine home and away team info into one DataFrame
players_info = pd.concat([
    home_team[['player_id', 'full_name']],
    away_team[['player_id', 'full_name']]
]).drop_duplicates().reset_index(drop=True)

# Merge event DataFrame with home_team and away_team DataFrames to get player IDs
event = event.merge(home_team[['match_id', 'player_id']], left_on='match_id', right_on='match_id', how='left', suffixes=('', '_home'))
event = event.rename(columns={'player_id': 'player_id_home'})  # Rename the merged column from home_team
event = event.merge(away_team[['match_id', 'player_id']], left_on='match_id', right_on='match_id', how='left', suffixes=('', '_away'))
event = event.rename(columns={'player_id': 'player_id_away'})  # Rename the merged column from away_team



# Map winner_code to actual player IDs
def get_winner_player_id(row):
    if row['winner_code'] == 1:
        return row['player_id_home']
    elif row['winner_code'] == 2:
        return row['player_id_away']
    else:
        return None

# Check if 'player_id_home' and 'player_id_away' columns are present
if 'player_id_home' in event.columns and 'player_id_away' in event.columns:
    event['winner_player_id'] = event.apply(get_winner_player_id, axis=1)
else:
    raise KeyError("Expected columns 'player_id_home' and 'player_id_away' not found in the event DataFrame.")

# Count the number of wins for each player using groupby
win_counts = event.groupby('winner_player_id').size().reset_index(name='win_count').rename(columns={'winner_player_id': 'player_id'})

print("Win Counts DataFrame:")
print(win_counts)

# Merge win counts with player information
player_wins = pd.merge(win_counts, players_info, on='player_id', how='left')

print("Player Wins DataFrame:")
print(player_wins)

# Find the player with the highest number of wins
if not player_wins.empty:
    top_player = player_wins.loc[player_wins['win_count'].idxmax()]
    print(f"Player with the highest number of wins: {top_player['full_name']} with {top_player['win_count']} wins")
else:
    print("No wins found.")


5.How many sets are typically played in a tennis match?

In [ ]:
home_team_score_df = grouped_data_frames['home_team_score']
# Select columns period_1 to period_5
period_columns = ['period_1', 'period_2', 'period_3', 'period_4', 'period_5']

# Count non-null values across rows (axis=1)
home_team_score_df['filled_columns_count'] = home_team_score_df[period_columns].count(axis=1)

# Find the mode of filled_sets_count
mode_sets_count = home_team_score_df['filled_columns_count'].mode().values[0]

print(f"Typically, {mode_sets_count} sets are played in a tennis match.")